In [9]:
import os 
import json
import pandas as pd
import traceback

In [10]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
KEY = os.getenv("OPENAI_API_KEY")


In [12]:
llm = ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature = 0.5)

In [13]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [14]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [15]:
TEMPLATE = """
TEXT:{text}
You are an expert MCQ maker. Given the above text,if it is your job to \
create a quiz of {number} multiple choice question for {subject} students in {tone} tone. 
Make sure the question are not repeated and check all the questions to be conforming the text as well. 
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}

"""

In [16]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template = TEMPLATE 
)

In [17]:
quiz_chain = LLMChain(llm = llm, prompt=quiz_generation_prompt,output_key="quiz",verbose = True)

In [18]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [19]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template = TEMPLATE 
)

In [20]:
review_chain = LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose = True)

In [21]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain], input_variables=["text","number","subject","tone","response_json"],
                                          output_variables = ["quiz","review"],verbose = True)

In [22]:
with open("D:\GenerativeAI\MCQGenerator\data.txt",'r') as data:
    TEXT = data.read()
    print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [23]:
# Serialise a python dictionary into a JSON formatted string
RESPONSE=json.dumps(RESPONSE_JSON)

In [24]:
SUBJECT="machine learning"
NUMBER= 5
TONE= "simple"

In [25]:
#tracking the token usage 
with get_openai_callback() as cb:
    response = generate_evaluate_chain({
        'text':TEXT,
        'number': NUMBER,
        'subject': SUBJECT,
        'tone':TONE,
        'response_json':RESPONSE
    })

d:\GenerativeAI\MCQGenerator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

TEXT:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [26]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1866
Prompt Tokens:1192
Completion Tokens:674
Total Cost:0.003136


In [27]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [28]:
quiz = response.get("quiz")

In [29]:
#json -> dictioanry
quiz_data = json.loads(quiz)

In [30]:
review = response.get("review")
json.loads(review)

{'1': {'mcq': 'What is biology?',
  'options': {'a': 'The study of rocks',
   'b': 'The study of life',
   'c': 'The study of stars',
   'd': 'The study of computers'},
  'correct': 'b'},
 '2': {'mcq': 'What is a major theme in biology that explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Chemistry',
   'c': 'Physics',
   'd': 'Mathematics'},
  'correct': 'a'},
 '3': {'mcq': 'What is the basic unit of life that processes hereditary information?',
  'options': {'a': 'Atoms', 'b': 'Molecules', 'c': 'Cells', 'd': 'Organs'},
  'correct': 'c'},
 '4': {'mcq': 'What is the process that allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Photosynthesis',
   'b': 'Respiration',
   'c': 'Energy processing',
   'd': 'Digestion'},
  'correct': 'c'},
 '5': {'mcq': 'What is the scientific method used by biologists to study the world around them?',
  'options': {'a': 'Guessing',
   'b': 'Intuition',
   'c': 'Logic',
   'd': 'Observation and experim

In [31]:
print(quiz)

{
"1": {
"mcq": "What is the scientific study of life called?",
"options": {"a": "Physics", "b": "Chemistry", "c": "Biology", "d": "Geology"},
"correct": "c"
},
"2": {
"mcq": "Which theme in biology explains the unity and diversity of life?",
"options": {"a": "Evolution", "b": "Photosynthesis", "c": "Mitosis", "d": "Respiration"},
"correct": "a"
},
"3": {
"mcq": "What allows organisms to move, grow, and reproduce?",
"options": {"a": "Energy processing", "b": "Cell division", "c": "Genetic mutation", "d": "Photosynthesis"},
"correct": "a"
},
"4": {
"mcq": "What is the process by which biologists make observations, generate hypotheses, and form conclusions?",
"options": {"a": "Scientific method", "b": "Trial and error", "c": "Guesswork", "d": "Intuition"},
"correct": "a"
},
"5": {
"mcq": "When did life on Earth emerge?",
"options": {"a": "1 million years ago", "b": "100 million years ago", "c": "1 billion years ago", "d": "3.7 billion years ago"},
"correct": "d"
}
}


In [32]:
quiz_table_data = []
for key,value in quiz_data.items():
    questions = value["mcq"]
    
    options = "|".join(
        [f"{option_key}: {option_value}"
          for option_key,option_value in value["options"].items()   
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"mcq":questions,"options":options,"correct":correct})
    
    



In [33]:
print(quiz_table_data)

[{'mcq': 'What is the scientific study of life called?', 'options': 'a: Physics|b: Chemistry|c: Biology|d: Geology', 'correct': 'c'}, {'mcq': 'Which theme in biology explains the unity and diversity of life?', 'options': 'a: Evolution|b: Photosynthesis|c: Mitosis|d: Respiration', 'correct': 'a'}, {'mcq': 'What allows organisms to move, grow, and reproduce?', 'options': 'a: Energy processing|b: Cell division|c: Genetic mutation|d: Photosynthesis', 'correct': 'a'}, {'mcq': 'What is the process by which biologists make observations, generate hypotheses, and form conclusions?', 'options': 'a: Scientific method|b: Trial and error|c: Guesswork|d: Intuition', 'correct': 'a'}, {'mcq': 'When did life on Earth emerge?', 'options': 'a: 1 million years ago|b: 100 million years ago|c: 1 billion years ago|d: 3.7 billion years ago', 'correct': 'd'}]


In [34]:
mcq_df=pd.DataFrame(quiz_table_data)

In [35]:
mcq_df.to_csv("mcq.csv",index=False)

In [36]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_21_2024_14_29_50'